In [ ]:
import pandas as pd
import requests
import numpy as np
import openpyxl
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
api_url='https://api.yelp.com/v3/businesses/search' #Nos dirigimos a la url de la API

#estos datos corresponden a una cuenta de usuario creada previamente
clientid='GWOCZh9-BmZxtdsAjr7Gug'
apikey='FHVvXoNmTXIl9DuxYis7AV5uLPujm9MLwrhgs5NgvCfaOxd3V6mxt6dQU8eEqYJiGxe816XATx7ufWjbMWqbV-2Uku1jxBJv8BGRC74NroLPl27PDQqs0tDixit-YHYx'
headers={'Authorization':'Bearer %s'%apikey}

In [ ]:
# Selecciono la ciudad que coincide con la que estamos trabajando en el dataset de restaurantes
city = "Houston"
 #Se obtiene la info de la API especificando la ciudad requerida mediante la variable 'city'
params={'term':'restaurants','location': city,'limit':50}
response=requests.get(api_url,params=params,headers=headers)
data=response.json()
data

{'businesses': [{'id': 'JNXmmqK4H5yJ1a9AfYTQ9g',
   'alias': 'credence-houston-3',
   'name': 'Credence',
   'image_url': 'https://s3-media0.fl.yelpcdn.com/bphoto/qGvSeSa3iz5__ecLKS1cuw/o.jpg',
   'is_closed': False,
   'url': 'https://www.yelp.com/biz/credence-houston-3?adjust_creative=GWOCZh9-BmZxtdsAjr7Gug&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_search&utm_source=GWOCZh9-BmZxtdsAjr7Gug',
   'review_count': 168,
   'categories': [{'alias': 'newamerican', 'title': 'New American'},
    {'alias': 'lounges', 'title': 'Lounges'}],
   'rating': 4.3,
   'coordinates': {'latitude': 29.782898, 'longitude': -95.53281},
   'transactions': ['pickup', 'delivery'],
   'location': {'address1': '9757 Katy Fwy',
    'address2': 'Ste 170',
    'address3': None,
    'city': 'Houston',
    'zip_code': '77024',
    'country': 'US',
    'state': 'TX',
    'display_address': ['9757 Katy Fwy', 'Ste 170', 'Houston, TX 77024']},
   'phone': '+17135682525',
   'display_phone': '(713) 568-2525',
   

In [ ]:
type(data)

dict

In [ ]:

offset=0

city = 'Houston'

rest_todos=[] #Lista para almacenar los DF de cada consulta

while offset<=150:


    #parámetros de la consulta a la API
    params={'term':'restaurants','location': city,'limit':50,'offset':offset}

    response=requests.get(api_url,params=params,headers=headers)
    data=response.json()

    print(len(data["businesses"]), "results found at offset", offset)

    data_businesses = data.get('businesses', [])
    if not data_businesses:
        print(f"No more results found at offset {offset}. Stopping the loop.")
        break

    #Aquí se obtienen los primeros 50 resultados. (50 primeros DataFrames)
    rests = pd.json_normalize(data["businesses"],sep='_',record_path=['categories'][0],meta=['id', 'name','price','rating','review_count','distance', ['coordinates','latitude'],['coordinates','longitude'],['location','address1']],errors='ignore')
    print("DataFrame created with", len(rests), "rows.")
    rests['city'] = city

    rest_todos.append(rests) #El DF resultante se agrega al final de la lista de DFs

    offset=offset+50 #Se agregan 50 resultados al ciclo.

50 results found at offset 0
DataFrame created with 112 rows.
50 results found at offset 50
DataFrame created with 118 rows.
50 results found at offset 100
DataFrame created with 129 rows.
50 results found at offset 150
DataFrame created with 121 rows.


Luego de realizar un ciclo que vaya generando los resultados de 50 en 50 según los parámetros, mostramos el resultado que ha sido igualado a la variable 'rest_todos'

In [ ]:
rest_todos

[            alias                  title                      id  \
 0     newamerican           New American  JNXmmqK4H5yJ1a9AfYTQ9g   
 1         lounges                Lounges  JNXmmqK4H5yJ1a9AfYTQ9g   
 2     newamerican           New American  c7_PCKjymhTEx-418TUQrw   
 3          venues  Venues & Event Spaces  2ntHBD8NPWAQwVTreiyo1g   
 4     musicvenues           Music Venues  2ntHBD8NPWAQwVTreiyo1g   
 ..            ...                    ...                     ...   
 107     wine_bars              Wine Bars  bRA-zzvte71rq5x45J7-zQ   
 108        diners                 Diners  bRA-zzvte71rq5x45J7-zQ   
 109    mideastern         Middle Eastern  ACqXOpwPvlv7r0B_bi6E5A   
 110         steak            Steakhouses  ACqXOpwPvlv7r0B_bi6E5A   
 111  tradamerican               American  ACqXOpwPvlv7r0B_bi6E5A   
 
                      name price rating review_count     distance  \
 0                Credence   NaN    4.3          168  7624.217144   
 1                Credence   NaN

A continuación, se crea un DF con  los resultados obtenidos

In [ ]:
rests=pd.concat(rest_todos,ignore_index=True)
rests

,alias,title,id,name,price,rating,review_count,distance,coordinates_latitude,coordinates_longitude,location_address1,city
0,newamerican,New American,JNXmmqK4H5yJ1a9AfYTQ9g,Credence,NaN,4.3,168,7624.217144,29.782898,-95.53281,9757 Katy Fwy,Houston
1,lounges,Lounges,JNXmmqK4H5yJ1a9AfYTQ9g,Credence,NaN,4.3,168,7624.217144,29.782898,-95.53281,9757 Katy Fwy,Houston
2,newamerican,New American,c7_PCKjymhTEx-418TUQrw,KP’s Kitchen,$$,4.6,298,3957.097593,29.785128,-95.493266,8412 Katy Fwy,Houston
3,venues,Venues & Event Spaces,2ntHBD8NPWAQwVTreiyo1g,The Rustic,$$,4.0,1014,1832.195484,29.75711,-95.456527,1121 Uptown Park Blvd,Houston
4,musicvenues,Music Venues,2ntHBD8NPWAQwVTreiyo1g,The Rustic,$$,4.0,1014,1832.195484,29.75711,-95.456527,1121 Uptown Park Blvd,Houston
...,...,...,...,...,...,...,...,...,...,...,...,...
475,szechuan,Szechuan,bRbVpY958-AaWLw92F--Vg,Duck N Bao,$$,4.1,270,3860.469001,29.76085,-95.41765,5535 Memorial Dr,Houston
476,southern,Southern,Az9zW6yZIp6Gr_yIeDr8HQ,The Moonshiners Southern Table + Bar,$$,3.7,818,9063.412169,29.76056,-95.36188,1000 Prairie St,Houston
477,cocktailbars,Cocktail Bars,Az9zW6yZIp6Gr_yIeDr8HQ,The Moonshiners Southern Table + Bar,$$,3.7,818,9063.412169,29.76056,-95.36188,1000 Prairie St,Houston
478,breakfast_brunch,Breakfast & Brunch,Az9zW6yZIp6Gr_yIeDr8HQ,The Moonshiners Southern Table + Bar,$$,3.7,818,9063.412169,29.76056,-95.36188,1000 Prairie St,Houston
